In [13]:
import pandas as pd

In [ ]:
dataset_name = 'mapbiomas_cobertura_col8'
raiz = '../database'

### Bronze

In [2]:
(pd.read_excel('https://storage.googleapis.com/mapbiomas-public/initiatives/brasil/collection_8/downloads/statistics/tabela_geral_mapbiomas_col8_biomas_municipios.xlsx',
               sheet_name = 1)
 .to_excel(f'{raiz}/bronze_{dataset_name}.xlsx',
           index=False)
)

### Fase Prata

|---------------------------------------------------------------------|
|                              COLEÇÃO 8 - CLASSES                              |
|-------------------------------------|---------------------------------|-------|
| Categoria                           | Subcategoria                    | ID    |
|-------------------------------------|---------------------------------|-------|
| 3.1. Pastagem                       |                                 | 15    |
| 3.2. Agricultura                    |                                 | 18    |
|                                     | 3.2.1. Lavoura Temporária      | 19    |
|                                     |     3.2.1.1. Soja             | 39    |
|                                     |     3.2.1.2. Cana             | 20    |
|                                     |     3.2.1.3. Arroz (beta)    | 40    |
|                                     |     3.2.1.4. Algodão (beta)  | 62    |
|                                     |     3.2.1.5. Outras Lavouras Temporárias | 41 |
|                                     | 3.2.2. Lavoura Perene          | 36    |
|                                     |     3.2.2.1. Café             | 46    |
|                                     |     3.2.2.2. Citrus           | 47    |
|                                     |     3.2.2.3. Dendê           | 35    |
|                                     |     3.2.1.3. Outras Lavouras Perenes | 48 |
| 3.4. Mosaico de Usos                |                                 | 21    |


Descrição dos códigos: https://brasil.mapbiomas.org/wp-content/uploads/sites/4/2023/08/Legenda-Colecao-8-LEGEND-CODE.pdf

In [2]:
anos = (pd.read_excel(f'{raiz}/bronze_{dataset_name}.xlsx',
               usecols=lambda x: isinstance(x, int))
        .columns
        .drop(range(1985, 2000))
        .tolist()
        )

In [14]:
df = (pd.read_excel(f'{raiz}/bronze_{dataset_name}.xlsx')
    .drop(range(1985, 2000), axis = 'columns')
    .query('class_id in (15, 18, 19, 39, 20, 40, 62, 41, 36, 46, 47, 35, 48, 21)') #https://brasil.mapbiomas.org/wp-content/uploads/sites/4/2023/08/Legenda-Colecao-8-LEGEND-CODE.pdf
    .assign(**{
        'uf': lambda df: (df['geocode']// 100000)
        }
            )
    .melt(
        id_vars=['uf', 'geocode', 'class_id'],
        value_vars=anos,
        var_name='ano',
        value_name='area_ha'
        )
    .dropna(subset='area_ha')
    .convert_dtypes()
    .rename(columns=
            {
                'geocode':'municipio',
                'class_id':'classe_solo'
             })
    .set_index(['uf', 'municipio', 'ano', 'classe_solo'])
    )

In [15]:
df.to_parquet(f'{raiz}/prata_mapbiomas_areaclass_agropecuaria.parquet')

In [16]:
(df.reset_index()
 .assign(**{
          'fonte': 'Deposição Atmosférica'
          })
 .groupby(['uf', 'municipio', 'ano', 'fonte'])
 .agg({'area_ha':sum})
 .to_parquet(f'{raiz}/prata_mapbiomas_areatotal_agropecuaria.parquet')
 )

### Fase Ouro

Área Destinada a Agropecuária (ha)
Elementos (kg/ha ano)

Cálculo: Cobertura Solo Municipal * (Parâmetro_X/1000)

UF e Brasil = mil t

NPK oriundo de deposição atmosférica incorporado ao solo (t)

In [17]:
parametro_N = 5.5
parametro_P = 0.3
parametro_K = 3.1

In [20]:
df = (pd.read_parquet(f'{raiz}/prata_areatotal_agropecuaria.parquet',
                      use_nullable_dtypes=True)
      .assign(**{
          'N': lambda df: df['area_ha']*(parametro_N/1000),
          'P': lambda df: df['area_ha']*(parametro_P/1000),
          'K': lambda df: df['area_ha']*(parametro_K/1000)
          })
      .drop('area_ha', axis = 1)
      .reset_index()
      .convert_dtypes()
      .melt(id_vars=['uf', 'ano', 'fonte'],
            value_vars=['N', 'P', 'K'],
            var_name='elemento',
            value_name='valor')
     .groupby(['uf', 'ano', 'fonte'])
     .agg({'valor':sum})
)

df.to_csv(f'{raiz}/ouro_areatotal_agropecuaria.csv')

In [19]:
df

valor
uf ano  fonte                              
11 2000 Deposição Atmosférica  45772.481071
   2001 Deposição Atmosférica   48163.71225
   2002 Deposição Atmosférica  51016.817704
   2003 Deposição Atmosférica  55005.538385
   2004 Deposição Atmosférica  58299.379717
...                                     ...
53 2018 Deposição Atmosférica   2398.165591
   2019 Deposição Atmosférica   2401.287179
   2020 Deposição Atmosférica   2403.828788
   2021 Deposição Atmosférica   2408.031639
   2022 Deposição Atmosférica   2371.963318

[621 rows x 1 columns]